In [26]:
import matplotlib.pyplot as plt
import numpy as np
import mxnet as mx
import os
import cv2
from mxnet import gluon, nd, image
from gluoncv.data import Kinetics400
from mxnet.gluon.data import DataLoader
from mxnet.gluon.data.vision import transforms
from gluoncv.data.transforms import video
from gluoncv import utils
from gluoncv.model_zoo import get_model
from gluoncv.utils.filesystem import import_try_install
import_try_install('decord')

<module 'decord' from '/opt/conda/lib/python3.7/site-packages/decord/__init__.py'>

In [35]:
# Load dataset into memory
transform_val = transforms.Compose([
    video.VideoCenterCrop(size=224),
    video.VideoToTensor()
])

val_dataset = Kinetics400(root=os.path.expanduser('./val/'), 
                          train=False,
                          transform=transform_val, 
                          video_loader=True, 
                          use_decord=True)
val_data = DataLoader(val_dataset, batch_size=8, shuffle=True)

In [36]:
for x, y in val_data:
    print('Video frame size (batch, extra, channel, depth, height, width):', x.shape)
    print('Video label:', y.shape)
    break

Video frame size (batch, extra, channel, depth, height, width): (8, 1, 3, 224, 224)
Video label: (8,)


In [54]:
import pandas as pd

val_dataset = pd.read_csv("~/.mxnet/datasets/kinetics400/kinetics400_train_list_rawframes.txt", 
                          header=None, 
                          delimiter=r"\s+",
                          names=["path", "frames", "class"])
val_dataset.drop(["frames"], axis=1, inplace=True)
271

val_dataset['path'] = '../val/' + val_dataset['path'].astype(str) + ".mp4"
val_dataset.to_csv('test.csv', header=False, index=False, sep=' ')

,path,class
0,../val/abseiling/0wR5jVB-WPk_000417_000427.mp4,0
1,../val/abseiling/3caPS4FHFF8_000036_000046.mp4,0
2,../val/abseiling/3yaoNwz99xM_000062_000072.mp4,0
3,../val/abseiling/6IbvOJxXnOo_000047_000057.mp4,0
4,../val/abseiling/6_4kjPiQr7w_000191_000201.mp4,0
...,...,...
2682,../val/cleaning_pool/8VU1bT5Zork_000215_000225...,62
2683,../val/cleaning_pool/AFvEYQkSmfk_000123_000133...,62
2684,../val/cleaning_pool/AHiu-Okbhho_000008_000018...,62
2685,../val/cleaning_pool/B8uJabL49FI_000074_000084...,62


In [ ]:
# Run this in the slowfast directory
python3 tools/run_net.py \
  --cfg configs/Kinetics/X3D_L.yaml \
  DATA.PATH_TO_DATA_DIR "../" \
  DATA.PATH_LABEL_SEPARATOR " " \
  DATA.DECODING_BACKEND "pyav" \
  TRAIN.ENABLE False \
  NUM_GPUS 1 \
  TEST.BATCH_SIZE 4